In [0]:
!pip install geopandas
!pip install fiona
!pip install descartes
!pip install mapclassify
# Librerías que se utilizó

In [0]:
import pandas as pd

In [0]:
data = pd.read_csv("https://raw.githubusercontent.com/ryali93/UNMSM_programacion/master/data/earthquakes.csv")

In [0]:
data.Longitude # Extraer la columna Longitude

In [0]:
data.Latitude

In [0]:
for i in zip(data.Longitude, data.Latitude ): # Con zip juntamos ambas columnas en columnas pero tuplas
  print(i)

In [0]:
from geopandas import GeoDataFrame # Extraer la función GeoDataFrame de la librería geopandas 

In [44]:
# En geometry cada tupla que será generado por el "for" será convertido en una geometría
# en este caso un punto "Point(n)" y será puesta dentro de una lista
# esto con el fin se insertarlo en un data frame 
geometry = [Point(n) for n in zip(data.Longitude, data.Latitude)] 

crs = {'init': 'epsg:4326'} # variable que contiene el codigo sist. coordenas de ref.

# Luego con la función GeoDataFrame creo un data frame con información espacial
# esa información espacial es los puntos (como tupla) insertado en un lista 
# y nombrado como geometry
geo_df = GeoDataFrame(data, crs=crs, geometry=geometry) # ojo: lo que está al lado de izquierdo del
                                                        # "=" (en cada argumento) es simplemente el nombre 
                                                        # del argumento
                                                        # lo que varía en sí es lo que está a la derecha  
                                                        # del "="
geo_df # Es similar a la tabla "data", pero ahora tiene un columna adicional: geometry (su info espacial)

,DateTime,Latitude,Longitude,Depth,Magnitude,MagType,NbStations,Gap,Distance,RMS,Source,EventID,geometry
0,2002/01/01 10:39:06.82,-55.2140,-129.0000,10.0,6.0,Mw,78.0,NaN,NaN,1.07,NEI,2.002010e+11,POINT (-129.00000 -55.21400)
1,2002/01/01 11:29:22.73,6.3030,125.6500,138.1,6.3,Mw,236.0,NaN,NaN,0.90,NEI,2.002010e+11,POINT (125.65000 6.30300)
2,2002/01/02 14:50:33.49,-17.9830,178.7440,665.8,6.2,Mw,215.0,NaN,NaN,1.08,NEI,2.002010e+11,POINT (178.74400 -17.98300)
3,2002/01/02 17:22:48.76,-17.6000,167.8560,21.0,7.2,Mw,427.0,NaN,NaN,0.90,NEI,2.002010e+11,POINT (167.85600 -17.60000)
4,2002/01/03 07:05:27.67,36.0880,70.6870,129.3,6.2,Mw,431.0,NaN,NaN,0.87,NEI,2.002010e+11,POINT (70.68700 36.08800)
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2408,2016/12/20 12:33:14.24,-10.1785,160.9149,10.0,6.0,Mw,NaN,14.0,1.0,0.88,us,2.016122e+11,POINT (160.91490 -10.17850)
2409,2016/12/21 00:17:14.99,-7.5082,127.9206,152.0,6.7,Mw,NaN,17.0,3.0,1.20,us,2.016122e+11,POINT (127.92060 -7.50820)
2410,2016/12/24 01:32:16.04,-5.2453,153.5754,35.0,6.0,Mw,NaN,13.0,2.0,0.91,us,2.016122e+11,POINT (153.57540 -5.24530)
2411,2016/12/25 14:22:27.05,-43.4029,-73.9395,38.0,7.6,Mw,NaN,29.0,0.0,0.80,us,2.016123e+11,POINT (-73.93950 -43.40290)


In [0]:
geo_df.geometry # Columna de la geometría

In [0]:
geo_df.plot() # Ahora como es una geometría + información alfanumerica
              # Es posible plotear esas geometrías que contiene cada fila de la tabla

In [0]:
# Lo que hemos realizado es un geoproceso que normalmente lo hacemos en SIG
# como Arcgis o Qgis, en este caso podemos visualizarlo en esas plataformas
# Por eso requerimos guardar la información procesada
geo_df.to_file(driver='ESRI Shapefile', filename='data.shp') # Se guarda en tu drive 
# donde 'data.shp' representa el nombre del archivo guardado
# driver='ESRI Shapefile' es el formato del archivo

In [0]:
# Ahora analizaremos el producto

In [0]:
import fiona # Librería que nos permite analizar datos vectoriales

In [0]:
c = fiona.open('data.shp')

In [58]:
print(c.driver) # Formato del vector
print(c.crs) # Codigo de sistema de referencia de coordenadas
print(len(c)) # Perimetro
print(c.bounds) # Extension limites

ESRI Shapefile
{'init': 'epsg:4326'}
2413
(-179.989, -65.651, 179.998, 84.948)


In [47]:
gjson = geo_df.to_crs(epsg='4326').to_json()
gjson

/usr/local/lib/python3.6/dist-packages/pyproj/crs.py:77: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method.
  return _prepare_from_string(" ".join(pjargs))


'{"type": "FeatureCollection", "features": [{"id": "0", "type": "Feature", "properties": {"DateTime": "2002/01/01 10:39:06.82", "Depth": 10.0, "Distance": null, "EventID": 200201014017.0, "Gap": null, "Latitude": -55.214, "Longitude": -129.0, "MagType": "Mw", "Magnitude": 6.0, "NbStations": 78.0, "RMS": 1.07, "Source": "NEI"}, "geometry": {"type": "Point", "coordinates": [-129.0, -55.214]}}, {"id": "1", "type": "Feature", "properties": {"DateTime": "2002/01/01 11:29:22.73", "Depth": 138.1, "Distance": null, "EventID": 200201014018.0, "Gap": null, "Latitude": 6.303, "Longitude": 125.65, "MagType": "Mw", "Magnitude": 6.3, "NbStations": 236.0, "RMS": 0.9, "Source": "NEI"}, "geometry": {"type": "Point", "coordinates": [125.65, 6.303]}}, {"id": "2", "type": "Feature", "properties": {"DateTime": "2002/01/02 14:50:33.49", "Depth": 665.8, "Distance": null, "EventID": 200201024034.0, "Gap": null, "Latitude": -17.983, "Longitude": 178.744, "MagType": "Mw", "Magnitude": 6.2, "NbStations": 215.0, 

In [0]:
# Ahora el siguiente proceso a realizar el ploteado de nuestro producto
# en un plataforma interactiva
# para ello, necesitamos transformar la tabla en un json
geo_df = geo_df.drop(columns=['DateTime']) # se elimina la columna 'DateTime'
                                           # aunque es opcional
                                           # si corres este codigo solo hazlo una vez
                                           # a la siquiente te saldrá error
gjson = geo_df.to_crs(epsg='4326').to_json() # luego la información que queda como 
                                             # geo_df la convertimos en un json
                                             # Un "diccionario metido en un string"

In [51]:
gjson

'{"type": "FeatureCollection", "features": [{"id": "0", "type": "Feature", "properties": {"Depth": 10.0, "Distance": null, "EventID": 200201014017.0, "Gap": null, "Latitude": -55.214, "Longitude": -129.0, "MagType": "Mw", "Magnitude": 6.0, "NbStations": 78.0, "RMS": 1.07, "Source": "NEI"}, "geometry": {"type": "Point", "coordinates": [-129.0, -55.214]}}, {"id": "1", "type": "Feature", "properties": {"Depth": 138.1, "Distance": null, "EventID": 200201014018.0, "Gap": null, "Latitude": 6.303, "Longitude": 125.65, "MagType": "Mw", "Magnitude": 6.3, "NbStations": 236.0, "RMS": 0.9, "Source": "NEI"}, "geometry": {"type": "Point", "coordinates": [125.65, 6.303]}}, {"id": "2", "type": "Feature", "properties": {"Depth": 665.8, "Distance": null, "EventID": 200201024034.0, "Gap": null, "Latitude": -17.983, "Longitude": 178.744, "MagType": "Mw", "Magnitude": 6.2, "NbStations": 215.0, "RMS": 1.08, "Source": "NEI"}, "geometry": {"type": "Point", "coordinates": [178.744, -17.983]}}, {"id": "3", "typ

In [0]:
import folium # Librería que nos permitira utilizar un mapa interactivo

In [0]:
# OJO : Puedo clasificar lo puntos antes que se conviertan a geojson
#       o sea cuando todavía es un geodataframe
# si quieres los terremos entre entre 8 y 9 de magnitud (por ejemplo)

In [53]:
# mapa representa la configuración basica del mapa (mapa base) pasa lo que esta abajo de los puntos
mapa = folium.Map([-8.783333, -74.866667], zoom_start=4, tiles='cartodbpositron')

points = folium.features.GeoJson(gjson) # Los puntos que serán ploteados

mapa.add_children(points) # Con add_children se addhiere los points encima de mapa base
mapa # ploteamos el mapa 

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: FutureWarning: Method `add_children` is deprecated. Please use `add_child` instead.
  """


In [0]:
3